 
 # Team 1 : Pierre Massimo Adamini, Valentin Belardi, Julian Vladimir José Ruiz Rodriguez, Zoé Frottier

## Table of content

- 1. Environment
- 2. Implementation
    - 2.1 Vision
    - 2.2 Path Planning
    - 2.3 Kindnapping
    - 2.4 Local Obstacle Avoidance 
    - 2.5 Kalman Filter
    - 2.6 Motion Control
    - 2.7 Global Implementation
- 3. Overall Project
- 4. Conclusion 
- 5. Sources

# 1. Environment

### Project Overview

**Firebot**, our robot, must navigate through a city to escape from a fire. The map consists of the following elements:

- **Buildings**: Represented as black rectangles. These structures act as obstacles.
- **Fire spots**: Represented as red rectangles. These are hazardous areas that the robot must avoid.
- **Debris**: Randomly appearing on the roads, acting as temporary obstacles.

### Objective

The robot's goal is to reach the **exit of the city**, indicated by a green square. To succeed, Firebot must:

- Maintain a safe distance from **fire spots**.
- Navigate around **buildings** and **debris** as needed. Passing close to these obstacles is acceptable, but proximity to fire should be avoided as much as possible.


# 2. Implementation



## 2.1 Vision

### Design Choices

For vision processing, we represent obstacles and the goal as **rectangles** on the global path, differentiated by their **colors**. This choice is driven by several factors:

1. **Rectangles offer reliable contours**:  
   - Rectangular shapes are less sensitive to noise compared to other polygonal shapes, ensuring consistent detection.  
   - As all obstacles share the same shape, a single set of parameters for the **Gaussian filter** and **Canny edge detection** is sufficient.

2. **Handling different shapes**:  
   - Our program focuses on rectangles for simplicity, but adding other polygonal shapes can be done without significant complexity.  
   - The robot (Thymio) is represented as a triangle
   - Local obstacles are white circles, minimizing the risk of confusing different elements.

3. **Color-based differentiation**:  
   - By leveraging both shape and color in OpenCV, we build a comprehensive vision system capable of distinguishing between map elements.



### Vision System Overview

The vision program consists of three main functions:
- **`sheet`**: Detects the map and defines its boundaries.  
- **`detect_obstacles`**: Identifies and classifies obstacles on the map.  
- **`detect_thymio`**: Detects the robot's position and orientation.  

Each function processes frames captured by the camera.



#### 1. Map Detection: `sheet`

The `sheet` function identifies the map, which is represented as a white rectangle on a dark background. The process involves:

1. **Preprocessing**:  
   - Convert the frame to grayscale.  
   - Apply a **Gaussian filter** to reduce noise.  
   - Use a threshold to create a binary image, enhancing contrast between the map and the background.

2. **Edge Detection**:  
   - Apply **Canny edge detection** to detect the map's boundaries.  
   - Since the map consists of straight rectangles, the orientation provided by Sobel or Prewitt filters is unnecessary, we therefore priviledge the accuracy and low sensitivity to noise of Canny Detection.

3. **Contour Detection**:  
   - Use OpenCV’s `findContours` function to extract contours.  
   - Select the largest contour, which corresponds to the map.
   - Remove all pixels outside the map's contour.

**Output**:  
The function returns the map’s top-left corner coordinates, height, and width in pixels.



#### 2. Obstacle Detection: `detect_obstacles`

The `detect_obstacles` function identifies rectangular obstacles on the map. The process involves:

1. **Edge Detection**:  
   - Similar to the `sheet` function, apply a **Gaussian filter** and **Canny edge detection**, but without thresholding to ensure lighter shades (e.g., light green) are detected.

2. **Contour Analysis**:  
   - Compute the **perimeter** of each contour.  
   - Approximate contours as polygons using OpenCV’s `approxPolyDP` with a tolerance of `0.04 * perimeter`, a value determined experimentally.

3. **Color Identification**:  
   - Create a mask where pixels inside the polygon are white.  
   - Use OpenCV’s `bitwise_and` function to isolate pixels within the polygon.  
   - Compute the **mean color** of these pixels.

4. **Filter Results**:  
   - Retain only black, green, and red rectangles.  
   - Exclude small red rectangles to avoid false positives from the robot's LEDs.

**Output**:  
The function returns a list of rectangles, each represented as a tuple containing the top-left and bottom-right corner coordinates (in pixels).



#### 3. Robot Detection: `detect_thymio`

The `detect_thymio` function uses the same process as `detect_obstacles`, focusing on black triangles. Once a triangle representing the robot is identified:

1. **Calculate Position**:  
   - Use trigonometric calculations to find the triangle's center.  

2. **Determine Orientation**:  
   - Compute the robot's orientation in radians based on its shape.

**Output**:  
The function returns the robot’s center coordinates and orientation.
To see the details of the calculations to obtain the center and the orientation, please refer to the comments on the vision code.


### Color Calibration

To classify colors (black, green, red), thresholds were determined empirically by testing various shades. This ensures compatibility with the camera under different lighting conditions.



### Summary
You can see below an example of the obsctacle and thymio detection together, with the margin of the thymio for the red and black obstacles. 

<img src="images/image_cam.jpeg" alt="camera" width="500">

The vision program effectively combines **shape detection** (rectangles, triangles, circles) and **color classification** to create a robust and adaptable system. By leveraging OpenCV functions like `Canny`, `findContours`, and `approxPolyDP`, we ensure accurate and efficient detection of all elements in the environment.


## 2.2 Path Planning

### Algorithm Implementation with Dijkstra's Algorithm

Once the vision system identifies the map's outline, obstacles, start point, and goal, the data is scaled to the size of the map and registered into tables. For this application, we decided to use **Dijkstra's algorithm**. To implement it, we divided the map space into a grid of cells, with each cell representing an area of $1 \text{cm}^2$. The map data is registered into this grid, where we compute two representations:  

1. **Character Grid**:  
   - `w` for walls (black obstacles).  
   - `f` for fire (red obstacles).  
   - `s` for the start point.  
   - `g` for the goal.  

2. **Numerical Grid**:  
   - Used for Dijkstra's algorithm computations.  

#### Map Design and Obstacles

The map includes two types of obstacles:  
- **Black walls**: Representing regular barriers.  
- **Red walls**: Representing fire zones, which require the robot (e.g., Thymio) to navigate with a greater margin.  

#### Algorithm Process

The algorithm processes the grid as follows:  
- **Grid incrementation**: Loops traverse the grid, assigning values in the numerical grid based on the distance of each cell from the start.  
- **Path Backtracking**: Once the goal is reached, the path is backtracked from the goal to the start by following the smallest numerical values to determine the shortest path.  

#### Special Handling for Fire Zones

To account for the features of fire walls, additional rules are implemented:  
- **Hazardous Zones**:  Cells directly adjacent to fire walls are marked as "hazardous" (`h`) and have a higher traversal cost.  
- **Dangerous Zones**:  Cells within a radius of 3 cm from fire walls are marked as "dangerous" (`d`).  
- **Traversal Cost**:  
  - `h` cells increment the traversal cost by 4 (compared to 1 for regular cells).
  - `d` cells increment the traversal cost by 2 (compared to 1 for regular cells).  

![grid_with_num.png](images/grid_with_num.png)


### Optimizing Path Points for Thymio Navigation

Initially, we decided to return all the points found in the path to the Thymio robot to compute its motion. However, we discovered that using such a large number of points was inefficient and caused several issues:  

- **Smoothness**: The motion was not smooth due to the excessive number of "reach points" along the trajectory.  
- **Error Recovery**: If the Thymio missed a checkpoint or diverged from the path, for local avoidance for example, it was difficult to determine at which point it should resume following the path.  

#### Solution

To address these issues, we decided to return only the points where the path changes direction. These points provide the most critical information while minimizing the number of required points, making it easier to reconstruct the complete path and navigate smoothly.

In the image below, the global path identified by the algorithm is shown in **light blue**, and the points used for navigation are highlighted in **blue cells**.


<img src="images/final_fig_glob_nav.png" alt="global_nav" width="1000">




## 2.3 Kidnapping


When the robot is **lifted off the ground**, it transitions into a **waiting state**, pausing its actions until it is placed back down. The robot uses its **ground sensor** to detect whether it is lifted or resting on the ground.

Once the robot is placed back in a **new random location**, it will:  
1. Wait to **recalculate its position**.  
2. **Recompute the global path** to the objective.  
3. Resume its journey toward the destination.  


## 2.4 Local Obstacle Avoidance

The robot enters the **Obstacle Avoidance** state when its front proximity sensors detect an obstacle within a specified distance. This state involves two key phases: **turning to avoid the obstacle** and **moving straight to clear the area**.

### 1. Turning to Avoid the Obstacle
- The robot uses an **artificial neural network** to process input from its front proximity sensors.  
- Each sensor value is assigned a weight, and the weighted sum determines the appropriate speed commands for the motors.  
- These weights were determined experimentally.  
- Due to latency between the USB dongle and the Thymio, a **delay (sleep)** is introduced, followed by motor deactivation within the same loop cycle.  
- This solution improves the precision of the robot’s turns, though it results in a **stepwise rotation** that is less smooth.

### 2. Moving Straight to Clear the Area
- After turning, the robot moves in a **straight line** to ensure it does not return to the obstacle, avoiding potential looping in front of it.

### Recovery and Path Resumption
After completing these steps, the robot:
1. **Retrieves its position**.  
2. Resumes the **global path** to its destination.  


## 2.5 Kalman Filter

### Choices 
To reduce uncertainty caused by the linearization of our model, we opted for a **non-linear model** and implemented an **Extended Kalman Filter (EKF)**.

### Model Definition 

The state vector of the system is defined as **[x, y, θ]**, representing the robot's position (x, y) and orientation (θ). The control input is defined as **[ωr, ωl]**, the angular velocities of the right and left wheels (in PWM), respectively. The model is then described by the following equations:

#### Prediction Equation
$$
x^+ = f(x,u)
$$
where:
$$
u = (\omega_r, \omega_l)
$$

The state transition is given by:
$$
\begin{pmatrix}
    x^+ \\
    y^+ \\
    \theta^+
\end{pmatrix}
=
\begin{pmatrix}
    x + T_s\cdot \cos(\theta)\frac{\omega_r + \omega_l}{2} \\
    y + T_s\cdot \sin(\theta)\frac{\omega_r + \omega_l}{2} \\
    \theta + T_s\frac{\omega_r - \omega_l}{L}
\end{pmatrix}
$$
where **T_s** is the sampling time and **L** is the distance between the wheels.

### Extended Kalman Filter

Since our model is non-linear, we use the **Extended Kalman Filter**. This requires the computation of the **Jacobian matrix** of the state function, which is given by:

$$
F(x) = \begin{pmatrix}
    1 & 0 & - T_s\cdot \sin(\theta)\frac{\omega_r + \omega_l}{2} \\
    0 & 1 &  T_s\cdot \cos(\theta)\frac{\omega_r + \omega_l}{2} \\
    0 & 0 & 1
\end{pmatrix}
$$

### State Estimation

With the Jacobian matrix, we can estimate the state of the robot using the Kalman filter. If camera measurements are available, we can update our state estimation based on them. Otherwise, we rely on the prediction made by the model.

### Noise and Tuning

To define the noise matrices **Q** (process noise) and **R** (measurement noise), we performed measurements of the Thymio's error and fine-tuned the values to best match the real noise in our environment.


## 2.6 Motion Control

### Controller Choice
We selected a **proportional controller** for motion control, as it sufficiently meets the needs of our application. Since the robot operates at moderate speeds, the complexity of incorporating differential or integral terms was unnecessary.

### Angle Calculation
<img src="images/angle.jpg" alt="angle" width="800">

This diagram illustrates the calculation of the robot's angles:

- The **position** and **angle** of the robot are provided by the Kalman filter.
- The **target position** is determined by the path planning algorithm.

The control is based on the difference between the robot's current angle ($\alpha$) and the target's angle ($\beta$), denoted as the angle $\gamma$.

Both $\alpha$ and $\beta$ are signed angles, measured in the trigonometric sense. The sign of the angle $\gamma$ determines the rotation direction:
- If $\gamma > 0$, the robot turns **left**.
- If $\gamma < 0$, the robot turns **right**.

Additionally, since the angle ranges from $+\pi$ to $-\pi$, we handle the transition by applying a modulo operation with $2\pi$ when the difference angle ($\gamma$) crosses this threshold.

### Controller Implementation
The **motion** function implements the controller. The core of the control logic is based on the difference between the robot's current angle and the target angle.

The control law is defined as:

$$
w_{control} = (angle_{target} - angle_{robot}) \cdot k_p
$$

Where:
- $w_{control}$ is the corrective control speed applied to the motors.
- $k_p$ is the proportional gain, determined experimentally.

We separate the control into two cases based on the angle difference:

1. **When the angle difference is below 35°**:  
   The robot moves forward at a constant speed, making small corrections to its trajectory.  
   $$ w_{right} = w_{const} + w_{control} $$  
   $$ w_{left} = w_{const} - w_{control} $$

2. **When the angle difference exceeds 35°**:  
   The robot turns solely to adjust its orientation.  
   $$ w_{right} = w_{control} $$  
   $$ w_{left} = -w_{control} $$

Where:
- $w_{right}$ is the PWM speed for the right motor.
- $w_{left}$ is the PWM speed for the left motor.

The value of **$k_p$** was determined through a combination of calculations and experimental tuning. The **35° angle threshold** was chosen as a reasonable value.

The **motion** function return the values of $w_{right}$ and $w_{left}$ for input of the kalman filter.



## 2.7 Global Implementation

The main code for the project is implemented in this Jupyter Notebook and is structured into several key parts. The **try, except, finally** structure ensures that the camera is properly turned off and the robot is stopped at the end of the program, whether it finishes normally or is interrupted. For performance reasons, we chose not to display the camera feed, as rendering the frame would increase execution time and negatively impact the program's performance.

### Code Structure
- **Vision, Path Planning, and Kalman Filtering**: These components are defined in separate Python files and imported into the Jupyter Notebook.
- **Obstacle Avoidance, Kidnapping,Motion Control and Main**: These parts are implemented directly within Jupyter notebook cells.
  
To enable communication with the Thymio robot, we use asynchronous communication. The command `await tdmclient.notebook.start()` starts the communication within the Jupyter notebook. Additionally, the `@tdmclient.notebook.sync_var` decorator is used for functions that interact with Thymio's variables, ensuring proper synchronization.

The functions that interact with Thymio’s variables—**Motion Control**, **Local Avoidance**, and **Kidnapping**—are defined directly within the notebook to be able to use the decorator.



### Initialization
- **Variable Initialization**: All necessary variables are initialized.
- **Initial Loop**: The first loop waits for the detection of the Thymio and the goal, which are the minimal conditions to start the project.
- **Path Calculation**: The path to be followed by the robot is calculated.



### Main Loop
In the main loop, the robot can be in one of three states:
1. **Kidnapped** (refer to the Kidnapping section).
2. **Avoid State** (refer to the Obstacle Avoidance section).
3. **Global Navigation State** (the default state).

The main loop continues executing until the robot reaches the goal, indicated by the boolean variable `arrive`.



### Global Navigation
- **Thymio Detection**: The system attempts to detect the Thymio. If detected, the position provided by the camera (in pixels) is converted to centimeters. If the Thymio is not detected, the variable `camera` is set to `None`, which is passed as input to the Kalman filter.
  
- **Execution Time**: The variables `start` and `end` are used to calculate the loop’s execution time. This time varies depending on whether the camera detects the Thymio or not. The execution time is used to update the `Ts` value in the Kalman filter.

- **Movement**: The robot moves towards its current target position, given by `path[path_step_index]`, using the `motion` function (see the Motion Control section).

- **Next Step Condition**: The robot progresses to the next target position when the difference between the current position and the next target is below another threshold. This threshold was also empirically determined.

- **Arrival Condition**: The robot determines that it has arrived at the goal when the difference between its current position and the goal is below a predefined threshold. This threshold was determined experimentally for optimal performance.





### Real-Time and Additional Plots
For visualizing the robot's current position along the path, we use **matplotlib** and **IPython.display**. These libraries allow the plot to be updated continuously at each loop iteration.

The plot includes:
- The **path**, represented by black dashed lines.
- The **current position of the robot**, indicated by a red cross and lines (from the Kalman filter).
- The **position from the camera**, represented by a blue square.
- The **variance of the robot's position**, shown as a red ellipse around the red cross (from the Kalman filter).
- The **goal position**, shown as a green square.

If the robot is kidnapped, the path is recalculated. The plot is cleared, and the new path is displayed.

Here is an example of the plot during execution. The horizontal axis represent the x position in cm and the vertical axis the y position in cm. The origin is in the top left corner.

<img src="images/plot_real_time.jpg" alt="angle" width="800">


The plots do not include obstacles, as implementing this feature was resource-intensive and would increase execution time, negatively impacting performance.


### Debugging and Data Logging
To assist with debugging, we include a variable called `DEBUG`. When `DEBUG` is set to `True`, various data points are saved, including:
- Kalman filter position.
- Camera position.
- Kalman variance.

This data are stored in tables, and additional cells can be used to generate further plots to provide more insight into the variables, which helps in debugging and performance analysis.

The additional plots available are : 

This figure  plots the values of position given by the kalman. It shows the position of the robot given by the kalman. The horizontal axis is the x positions and the vertical axis is the y positions. The axis are in cm and the origin is in the top left corner.

<img src="images/kalman_pos.jpg" alt="angle" width="500">


This figure plots the values of the position of the robot given by the kalman. The horizontal axis represent the timestep and the vertical axis the position. The blue curve is the x position and the orange curve is the y position. 


<img src="images/plot_pos.png" alt="angle" width="500">


This figure shows the variances of the x_position (in blue), y position (in red) and the robot angle (in green). The horizontal axis represent the timestep and the vertical axis the value of the variances given by the kalman filter. We can see that the variance (elipses) increases slowly when we remove the camera.


<img src="images/variance2.jpeg" alt="angle" width="500">







# 3. Overall Project

In [1]:
# Import tdmclient Notebook environment
import tdmclient.notebook
await tdmclient.notebook.start()
#Librairy import
import numpy as np
import cv2
import time
import matplotlib.pyplot as plt 


#import of global_nav, vision and filter functions
from global_nav_V2 import global_navigation_algorithm
from clean_vision import detect_thymio, detect_obstacles, sheet
from new_kalman_filter import kalman_filter
from IPython.display import clear_output
from matplotlib.patches import Ellipse

In [ ]:
#MOTION CONTROL CODE


ANGLE_DIFFERENCE=0.6


@tdmclient.notebook.sync_var
def avance(PWM_R,PWM_L):
    '''
    Set the target speed to the motor

    parameters : 
    - PWM_R: speed in PWM of the right motor
    - PWM_L: speed in PWM of the left motor
    
    '''
    global motor_left_target, motor_right_target
    motor_left_target=PWM_L
    motor_right_target=PWM_R
    return
@tdmclient.notebook.sync_var
def get_speed():
    '''
    return the actual speed of the motor 
    '''

    global motor_left_speed,motor_right_speed
    return motor_left_speed,motor_right_speed

def motion(pos_cible,pos_initial):
    '''
    P controller, calculate the speed of the motor to go to a position 
    parameters: 
    pos_cible: target position [x,y] [cm]
    pos_initial: actual_position of the robot and angle: [[x],[y],[alpha]] [cm, rad]
    '''
    kp_theta=1.4
    W_const=100

    angle_dir=-np.atan2((pos_cible[1]-round(pos_initial[1][0])),(pos_cible[0]-round(pos_initial[0][0])))

    #control wich depedend on the difference between the target angle et the robot angle
    angle_diff = angle_dir-pos_initial[2][0]

    # if the difference is too high, we need to modulate the angle_diff
    if angle_diff>np.pi:
        angle_diff=angle_diff-2*np.pi
    if angle_diff<-np.pi:
        angle_diff=angle_diff+2*np.pi

    w_control= kp_theta*(angle_diff)*180/np.pi

    #if the angle difference is too high : turn only
    if abs(angle_diff)>ANGLE_DIFFERENCE:
        w_r = int(w_control)
        w_l = int(-w_control)
    #if the angle difference is small : go forward and turn 
    else:
        w_r = int(W_const + w_control )
        w_l = int(W_const - w_control )
    
    avance(w_r,w_l)
   
    return w_r, w_l


In [ ]:
#LOCAL AVOIDANCE  CODE


@tdmclient.notebook.sync_var
def prox_detection():
    """
    Function to detect proximity obstacles.
    Returns True if an obstacle is detected within a specified range.
    """
    trsld_min = 2500 # rang of detection
    global prox_horizontal
    x = [0] * (len(prox_horizontal)-2)
    for i in range(len(x)):
        if prox_horizontal[i] > trsld_min:
            x[i] = prox_horizontal[i]
            return True
    return False


@tdmclient.notebook.sync_var
def motor_str(speed):
    """
    Sets both motors to the same speed to move the robot straight when into local avoidance state.
    Args:
        speed : Speed value which is sent to the motors.
    """
    global motor_left_target, motor_right_target
    motor_left_target = speed
    motor_right_target = speed
    return

@tdmclient.notebook.sync_var
def motor_turn(speed_L, speed_R):
    """
    Sets the left and right motor speeds to different values for turning the robot when into local avoidance state.
    Args:
        speed_L : Speed value which is sent to the left motor.
        speed_R : Speed value which is sent to the right motor.
    """
    global motor_left_target, motor_right_target
    motor_left_target = speed_L
    motor_right_target = speed_R
    return


@tdmclient.notebook.sync_var
def obst_avoid():
    """
    Implements the obstacle avoidance state for the thymio using the proximity sensors.
        - Uses weighted sensor values to compute motor speeds for obstacle avoidance.
        - Continues until obstacles are no longer detected and the robot has moved straight
          for a specified duration.
    """
    global prox_horizontal, motor_left_target, motor_right_target, state, not_str, time_front
    #Local avoidance constants initialisation
    # Weight of sensors
    w_l = [40,  20, -20, -20, -40]
    w_r = [-40, -20, -20,  20,  40]
    sensor_scale = 2500 #scale the wait for the command of the motors

    # Stop the motors
    motor_str(0)
    # Variables pour les calculs
    y = [0, 0]
    x = [0] * (len(prox_horizontal)-2)

    exit_avoid = False #Bool to exit avoid state
    str_counter = 0 #Counter for the straight state
    while not exit_avoid:
        x = prox_horizontal
        prox_pres = prox_detection()
        if prox_pres:
            str_counter = 0
            for i in range(len(x)-2):
                y[0] += x[i] * w_l[i] //sensor_scale
                y[1] += x[i] * w_r[i] //sensor_scale
            motor_turn(y[0],y[1]) #Turn the robot until no more obstacle is detected.
            time.sleep(0.15)
            motor_str(0)
        else:
            motor_str(200) #Go straight when no obstacle is detected anymore
            str_counter += 1
            if str_counter > 5:
                exit_avoid = True #Bool to exit the local avoidance state
        time.sleep(0.15)
    motor_str(0)
    not_str = 0
    return

In [ ]:
GROUND_THRESHOLD=500

@tdmclient.notebook.sync_var
def ground_detect():
    """
    Detects if the robot is on the ground. 
    Returns:
            - `True` if the ground sensor return a value lower than 500.
            - `False` if the ground sensor return a value higher than 500.
    """
    global prox_ground_reflected
    if prox_ground_reflected[0] >= GROUND_THRESHOLD:
        return False
    return True

In [ ]:
#Main Code

#COnstants
ARRIVAL_DISTANCE=5
STEP_DISTANCE=3
DEBUG=True
#varaible to save data to make plot
kalman_pos = []
camera_pos = []
kalman_P_x = []
kalman_P_y = []
kalman_P_theta = []

def main():
    
    print("Début du programme")

    #constantes 
    WIDTH_CM=107
    HEIGTH_CM=72

    #initialisation variable 
    cap = cv2.VideoCapture(0)
    pos_thymio = None
    green_rectangles = []
    x,y,w,h = 0,0,0,0
    #bool for arrival 
    arrive=False
    #step of the path
    path_step_index=0
    #bool for obstacle detected 
    avoid_state = False
    #save the position of the thymio
    x_data = []
    y_data = []
    #initalization of the plot
    fig, ax = plt.subplots(figsize=(15, 10))
    line, = ax.plot([], [], '-x', color='red')

    #scale axis: x axis is 107 cm, y axis is 72 cm c onvert in m
    ax.set_xlim(0,WIDTH_CM)
    ax.set_ylim(0,HEIGTH_CM)
    ax.invert_yaxis()

    #display the plot in real time
    display(fig)
    
    
    try: 
        #Initialization, wait for the detection of the thymio and the goal
        while pos_thymio == None or len(green_rectangles) == 0:
            ret, frame = cap.read()
            if not ret:
                break
            x,y,w,h = sheet(frame, cap)
            frame = frame[y:y+h, x:x+w]
            _, black_rectangles, green_rectangles, red_rectangles = detect_obstacles(frame)
            _, pos_thymio, angle_thymio = detect_thymio(frame)

            print("thymio detected ",pos_thymio)
            print("goal detected ",green_rectangles)

        #calculate the path
        path=global_navigation_algorithm(black_rectangles, pos_thymio, green_rectangles, red_rectangles, h, w)

        #conversion from pixel to cm
        pos_thymio_cm = np.array(pos_thymio)    
        pos_thymio_cm[0] = (pos_thymio[0] / w) * WIDTH_CM
        pos_thymio_cm[1] = (pos_thymio[1] / h) * HEIGTH_CM

        #initialization of the kalman variables
        pos_thymio_est=np.array([[pos_thymio_cm[0]],[pos_thymio_cm[1]], [angle_thymio]])
        P_est=1000*np.ones(3)
        #output of the control 
        wr = 0
        wl = 0

        #count the lost of the camera
        counter=0
        #intialization of the timer for the kalman 
        start = time.time()

        #while the robot is not at the goal
        while(not(arrive)):
            
            #detect if there is an obstacle
            avoid_state = prox_detection()
            #detect if it is kidnapped
            kidnapped = ground_detect()


            # ============  KIDNAPPING =================
            if kidnapped:
                avance(0,0)
                print("kidnapped")
                time.sleep(3)
                pos_thymio = None
                #wait to find the thymio
                while pos_thymio == None :
                    print("recherche du thymio")
                    ret, frame = cap.read()
                    if not ret:
                        break
                    frame = frame[y:y+h, x:x+w]
                    _,pos_thymio,_=detect_thymio(frame)

                #recalculate the path from the new position 
                path=global_navigation_algorithm(black_rectangles, pos_thymio, green_rectangles, red_rectangles, h, w)
                
                #reinitialization of the step of the path 
                path_step_index = 0

                # clear the real time plot
                ax.cla()
                line, = ax.plot([], [], '-x', color='red')
                x_data = []
                y_data = []
                ax.set_xlim(0,WIDTH_CM)
                ax.set_ylim(0,HEIGTH_CM)
                ax.invert_yaxis()


             # ============  LOCAL AVOIDANCE =================
            elif avoid_state:
                obst_avoid()

                #wait to find the thymio
                while pos_thymio == None :
                    print("recherche du thymio")
                    ret, frame = cap.read()
                    if not ret:
                        break
                    frame = frame[y:y+h, x:x+w]
                    _,pos_thymio,_=detect_thymio(frame)

            # ============ GLOBAL NAVIGATION =================
            else:
                _,frame=cap.read()
                frame = frame[y:y+h, x:x+w]
                _,pos_robot_cam,angle_robot_cam=detect_thymio(frame)  

                #Update of the position 
                if pos_robot_cam is None:
                    camera = None
                    counter += 1
                else:
                    #conversion pixel to cm
                    new_pos_thymio = np.array(pos_robot_cam)
                    new_pos_thymio[0] = (float(pos_robot_cam[0]) / w) * WIDTH_CM
                    new_pos_thymio[1] = (float(pos_robot_cam[1]) / h) * HEIGTH_CM
                    camera=np.array([[new_pos_thymio[0]],[new_pos_thymio[1]], [angle_robot_cam]])
                    #add the position saw by the camera on the plot (blue square)
                    ax.plot(new_pos_thymio[0], new_pos_thymio[1], 'ks', color='blue')

                
            
                #timer for the execution time between to call of the kalman 
                end = time.time()
                delta = end-start
                #call kalman function 
                pos_thymio_est, P_est  = kalman_filter(wr, wl, camera,pos_thymio_est,P_est, delta) 

                #start the timer for the execution time between to call of the kalman 
                start = time.time()
            
                
                #arrival condition 
                if (abs(pos_thymio_est[0]-path[-1][0]) <= ARRIVAL_DISTANCE and abs(pos_thymio_est[1]-path[-1][1]) <= ARRIVAL_DISTANCE):
                    arrive=True
                    break
                #condition for the next step
                if (abs(pos_thymio_est[0]-path[path_step_index][0]) <= STEP_DISTANCE and abs(pos_thymio_est[1]-path[path_step_index][1]) <= STEP_DISTANCE):
                    path_step_index=path_step_index+1
                # call motion function, take 
                wr, wl = motion(path[path_step_index],pos_thymio_est)

                if DEBUG:
                    #save data for the plot
                    kalman_pos.append(pos_thymio_est)
                    kalman_P_x.append(P_est[0][0])
                    kalman_P_y.append(P_est[1][1])
                    kalman_P_theta.append(P_est[2][2])

                #sleep to let the time for variables updates
                # time.sleep(0.1)

                # ================== PLOT ==================
                
                #plot the variance of the position from the kalman in ellipse
                ellipse = Ellipse((pos_thymio_est[0], pos_thymio_est[1]), width=2*P_est[0][0], height=2*P_est[1][1], edgecolor='r', facecolor='none')
                ax.add_patch(ellipse)
                ax.plot([i[0] for i in path], [i[1] for i in path], '--', color='black')
                ax.plot(path[-1][0], path[-1][1], 'ks', color='green')
                x_data.append(pos_thymio_est[0])
                y_data.append(pos_thymio_est[1])

                #update plot the position of the thymio in real time(red cross)
                line.set_xdata(x_data)
                line.set_ydata(y_data)

                clear_output(wait=True)
                display(fig)

        print("Arrivé!!!!!")
        avance(0,0)

    except KeyboardInterrupt:
            print("\n Keyboard interruption")

    finally:
        #close the windows and the camera
        cap.release()
        cv2.destroyAllWindows()
        print("Caméra fermée.")
        print("nombre de frames sans thymio détecté: ", counter)
        avance(0,0)

#call function main
main()

In [ ]:
#Additional plot for debbuging 

if(DEBUG):


    plt.plot([x[0][0] for x in kalman_pos], [x[1][0] for x in kalman_pos], 'b', label='Path')
    plt.legend()

    #scale axis: x axis is 107 cm, y axis is 72 cm
    plt.xlim(0,107)
    plt.ylim(0,72)
    plt.gca().invert_yaxis()
    plt.show()

    plt.plot(kalman_P_x, 'b', label='P_x')
    plt.plot(kalman_P_y, 'r', label='P_y')
    plt.plot(kalman_P_theta, 'g', label='P_theta')
    plt.legend()
    plt.show()

    plt.plot([x[0] for x in kalman_pos], label='Position X')
    plt.plot([x[1] for x in kalman_pos], label='Position Y')
    plt.legend()

    plt.show()




# 4. Conclusion

In this project, we developed an application that enables a robot to autonomously navigate a map with both fixed obstacles of various types and local obstacles that may appear during operation. The robot is designed to handle situations such as being "kidnapped" or having a lack of vision.

## Challenges
One of the main challenges we faced was managing the robot's **real-time performance**. Wireless communication introduces latency, which we had to account for in our system. 

Another challenge was **camera calibration**, as the camera is highly sensitive to lighting conditions and required careful tuning.

The project also involved extensive **parameter tuning**, including:
- Tuning the **proportional controller** (the **$k_p$** term),
- Adjusting the **Kalman filter noise parameters**,
- Setting appropriate thresholds for the **camera detection** and **obstacle avoidance**.

## Limitations and Improvements
We opted for a simplified approach to **path planning**(as detailed in path plannig section), focusing only on direction changes. As a result, the robot can experience slight deviations between two points. Moreover, we set the **obstacle avoidance threshold** large enough to ensure the robot avoids obstacles without colliding, but this aspect could be further optimized for smoother operation.

Initially, we designed the algorithm for a smaller grid (30x30 cells). However, as the project evolved, we ended up increasing the amount of cells in the grid, which reduced the performance of the pathfinding algorithm. Although the performance remained acceptable for our application this experience highlighted the limitations of such algorithm and the importance of choosing the right algorithm based on application-specific parameters.

Overall, this project allowed us to gain a deeper understanding of key robotics concepts, including:
- **Computer vision**,
- **Filtering techniques** (Kalman filter),
- **Path planning**,
- **Sensor detection**,
- **Motor control**.


# 5. Sources

We would like to mention that all the text and information found in the report was wrote by ourselves and we used Chat GPT as a tool to correct the grammar and the spelling as well as to improve readability. No additional content was generated.

### Vision : 
Examples of OpenCV of the Mobile Robotics class
https://docs.opencv.org/3.4/d2/d96/tutorial_py_table_of_contents_imgproc.html
https://docs.opencv.org/3.4/d7/d16/tutorial_py_table_of_contents_core.html
https://docs.opencv.org/3.4/d8/d23/classcv_1_1Moments.html

### Kalman : 
Exercices and theory of Filters of the Mobile Robotics class

https://en.wikipedia.org/wiki/Extended_Kalman_filter

### Path plannig : 
Lecture 4/5 on navigation 

## Main 
Control Your Thymio

## Local Avoidance
Obstacle Avoidance exercices and theory of the Mobile Robotics class


